In [ ]:
#!pip install 'matplotlib>=3.4' hist

In [ ]:
from collections import defaultdict
import pickle
import matplotlib.pyplot as plt
import hist
import numpy as np

In [ ]:
with open("eval_results_10k.pickle", "rb") as f:
    stats = pickle.load(f)

In [ ]:
HIST, OCCH, MEANS, OCCS = (0, 1, 2, 3)

In [ ]:
default_opts = dict(linestyle="--", linewidth=1)
plot_opts = defaultdict(lambda: default_opts)
plot_opts["real"] = dict(linewidth=1.5)
plot_opts["IEAGAN"] = dict(linewidth=1.5)

In [ ]:
legend_names = {
    "real": "GEANT4 (reference)",
    "IEAGAN": "IEA-GAN",
    "BigGAN_deep": "BigGAN-deep",
    "ContraGAN": "ContraGAN",
    "PEGAN": "PE-GAN"
}

In [ ]:
def plot_occh(ax):
    for key in stats:
        h = hist.Hist(stats[key][OCCH])
        ax.stairs(h.density(), h.axes.edges[0], label=legend_names[key], **plot_opts[key])
        if key == "real":
            ax.stairs(h.density(), h.axes.edges[0], fill=True, color="Gray", alpha=0.2)
    ax.set_ylabel("Density")
    ax.set_xlabel("Occupancy")

In [ ]:
fig, ax = plt.subplots()
plot_occh(ax)
ax.legend()
plt.gcf().set_size_inches(6,4.32)
fig.tight_layout()
fig.savefig("occupancy_distribution.pdf")

In [ ]:
def density(h):
    "density that includes flow bins (h.density() does not include it)"
    fraction = h.values() / h.sum(flow=True)
    return fraction / h.axes[0].widths

In [ ]:
def plot_hist(ax, bin_selection=lambda h: h[1:100]):
    for key in stats:
        h = hist.Hist(stats[key][HIST])
        h = bin_selection(h)
        ax.stairs(density(h), h.axes.edges[0], label=legend_names[key], **plot_opts[key])
        if key == "real":
            ax.stairs(density(h), h.axes.edges[0], fill=True, color="Gray", alpha=0.2)
    ax.set_ylabel("Density")
    ax.set_xlabel("Pixel Intensity")

In [ ]:
fig, ax = plt.subplots()
plot_hist(ax)
ax.legend()
plt.gcf().set_size_inches(6,4.32)
fig.tight_layout()
fig.savefig("pixel_intensity_lin.pdf")

In [ ]:
fig, ax = plt.subplots()
plot_hist(ax, bin_selection=lambda h: h[1:])
ax.set_yscale("log")
ax.legend()
ax.set_ylim(0.5e-10, 1e-3)
plt.gcf().set_size_inches(6,4.32)
fig.tight_layout()
fig.savefig("pixel_intensity_log.pdf")

In [ ]:
def plot_occs(ax):
    for key in stats:
        x = np.arange(41)
        y = stats[key][OCCS]
        y = np.append(y, y[-1])
        plt.step(x, y, where="post", label=legend_names[key], **plot_opts[key])
        if key == "real":
            plt.fill_between(x, y, step="post", color="Gray", alpha=0.2)
    ax.set_ylabel("Mean Occupancy")
    ax.set_xlabel("Sensor ID")
    ax.axvline(16, color="black", linestyle="--")
    ax.text(2, 0.008, "Inner detector layer")
    ax.text(22, 0.008, "Outer detector layer")

In [ ]:
fig, ax = plt.subplots()
plot_occs(ax)
ax.legend()
plt.gcf().set_size_inches(6,4.32)
fig.tight_layout()
fig.savefig("mean_occupancy.pdf")

In [ ]:
fig, [[ax1, ax2], [ax3, ax4]] = plt.subplots(nrows=2, ncols=2)
plot_hist(ax1)
plot_hist(ax2, bin_selection=lambda h: h[1:])
ax2.set_yscale("log")
plot_occh(ax3)
plot_occs(ax4)
ax4.set_ylim(0, 0.02)
fig.legend(*ax1.get_legend_handles_labels(),  loc='upper center', borderaxespad=0., ncol=5, bbox_to_anchor=(0., 0.83, 1., .102))
plt.gcf().set_size_inches(6,4.32)
fig.tight_layout()
fig.savefig("pixel_level_metrics.pdf")
plt.subplots_adjust(
    #left=0.1,
    #bottom=0.1, 
    #right=0.9, 
    #top=0.9, 
    #wspace=0.4, 
    hspace=0.4
)